In [2]:
import numpy as np
import pandas as pd
from tsfresh import extract_features, extract_relevant_features
pd.set_option("display.max_columns", 400)
def feature_row(book):
    for i in [
            1,
            2,
    ]:
        # wap
        book[f'book_wap{i}'] = (book[f'bid_price{i}'] * book[f'ask_size{i}'] +
                                book[f'ask_price{i}'] *
                                book[f'bid_size{i}']) / (book[f'bid_size{i}'] +
                                                         book[f'ask_size{i}'])

    # mean wap
    book['book_wap_mean'] = (book['book_wap1'] + book['book_wap2']) / 2

    # wap diff
    book['book_wap_diff'] = book['book_wap1'] - book['book_wap2']

    # other orderbook features
    book['book_price_spread'] = (book['ask_price1'] - book['bid_price1']) / (
        book['ask_price1'] + book['bid_price1'])
    book['book_bid_spread'] = book['bid_price1'] - book['bid_price2']
    book['book_ask_spread'] = book['ask_price1'] - book['ask_price2']
    book['book_total_volume'] = book['ask_size1'] + book['ask_size2'] + book[
        'bid_size1'] + book['bid_size2']
    book['book_volume_imbalance'] = (book['ask_size1'] + book['ask_size2']) - (
        book['bid_size1'] + book['bid_size2'])
    return book

In [7]:
book0 = pd.read_parquet('../input/book-ffill/book_train.parquet/stock_id=0/8a63186613aa46f98aeaa288e982be5a.parquet')
trade0 = pd.read_parquet('../input/trade_ffill/trade_train.parquet/stock_id=0/0788f1b95ece49f58ac27c87ca6b5fa7.parquet')
df0 = book0.merge(trade0, on=['time_id', 'seconds_in_bucket'])
df0 = feature_row(df0)
df0

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,price,size,order_count,book_wap1,book_wap2,book_wap_mean,book_wap_diff,book_price_spread,book_bid_spread,book_ask_spread,book_total_volume,book_volume_imbalance
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,1.002301,0,0,1.001434,1.001390,1.001412,0.000044,0.000439,0.000052,-0.000052,331,321
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,1.002301,0,0,1.001448,1.001390,1.001419,0.000058,0.000439,0.000052,-0.000052,205,195
2,5,2,1.001422,1.002301,1.001370,1.002353,3,100,2,100,1.002301,0,0,1.001448,1.001390,1.001419,0.000058,0.000439,0.000052,-0.000052,205,195
3,5,3,1.001422,1.002301,1.001370,1.002353,3,100,2,100,1.002301,0,0,1.001448,1.001390,1.001419,0.000058,0.000439,0.000052,-0.000052,205,195
4,5,4,1.001422,1.002301,1.001370,1.002353,3,100,2,100,1.002301,0,0,1.001448,1.001390,1.001419,0.000058,0.000439,0.000052,-0.000052,205,195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297995,32767,595,0.998275,0.998754,0.998179,0.998946,92,90,26,28,0.998547,0,0,0.998517,0.998549,0.998533,-0.000031,0.000240,0.000096,-0.000192,236,0
2297996,32767,596,0.998275,0.998754,0.998179,0.998946,92,90,26,28,0.998547,0,0,0.998517,0.998549,0.998533,-0.000031,0.000240,0.000096,-0.000192,236,0
2297997,32767,597,0.998275,0.998754,0.998179,0.998946,92,90,26,28,0.998547,0,0,0.998517,0.998549,0.998533,-0.000031,0.000240,0.000096,-0.000192,236,0
2297998,32767,598,0.998275,0.998754,0.998179,0.998946,92,90,26,28,0.998547,0,0,0.998517,0.998549,0.998533,-0.000031,0.000240,0.000096,-0.000192,236,0


In [8]:
y = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
y0 = y.loc[y['stock_id']==0]
y0 = pd.Series(y0.target.values, index=y0.time_id)
y0

time_id
5        0.004136
11       0.001445
16       0.002168
31       0.002195
62       0.001747
           ...   
32751    0.002611
32753    0.001190
32758    0.004264
32763    0.004352
32767    0.001084
Length: 3830, dtype: float64

In [10]:
extracted_features = extract_relevant_features(df0[['book_wap_mean', 'time_id', 'seconds_in_bucket']], y0, column_id="time_id", column_sort="seconds_in_bucket")
extracted_features.to_csv('data/tsfresh_extracted_features.csv', index=False)

Feature Extraction: 100%|██████████| 15/15 [27:48<00:00, 111.21s/it] 


In [11]:
extracted_features

,"book_wap_mean__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2","book_wap_mean__fft_coefficient__attr_""abs""__coeff_60","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.4","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.4","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.4","book_wap_mean__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.4","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","book_wap_mean__fft_coefficient__attr_""abs""__coeff_83","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.0","book_wap_mean__fft_coefficient__attr_""abs""__coeff_19",book_wap_mean__spkt_welch_density__coeff_8,book_wap_mean__spkt_welch_density__coeff_5,book_wap_mean__spkt_welch_density__coeff_2,"book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0","book_wap_mean__fft_coefficient__attr_""abs""__coeff_29","book_wap_mean__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0","book_wap_mean__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""mean""","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","book_wap_mean__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.6","book_wap_mean__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","book_wap_mean__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""",book_wap_mean__variation_coefficient,book_wap_mean__variance,"book_wap_mean__fft_aggregated__aggtype_""centroid""","book_wap_mean__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","book_wap_mean__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""var""","book_wap_mean__fft_coefficient__attr_""abs""__coeff_93","book_wap_mean__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""",book_wap_mean__absolute_sum_of

In [12]:
y0

time_id
5        0.004136
11       0.001445
16       0.002168
31       0.002195
62       0.001747
           ...   
32751    0.002611
32753    0.001190
32758    0.004264
32763    0.004352
32767    0.001084
Length: 3830, dtype: float64

In [20]:
def fix_jsonerr(df):
    df.columns = ["".join(c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
    return df
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
extracted_features = fix_jsonerr(extracted_features)
x_train = extracted_features.iloc[:int(0.8*len(extracted_features))]
x_val = extracted_features.iloc[int(0.8*len(extracted_features)):]
y_train = y0.iloc[:int(0.8*len(extracted_features))]
y_val = y0.iloc[int(0.8*len(extracted_features)):]

In [21]:
from lightgbm import LGBMRegressor
params = {
    'random_state': 42,
    'objective': 'rmse',  
    'boosting_type': 'gbdt',
    'min_child_samples': int(0.01*x_train.shape[0])
}
model = LGBMRegressor(**params)
model.fit(x_train, 
          y_train, 
          sample_weight=train_weights,
          eval_set=[(x_val, y_val)],
          eval_sample_weight=[val_weights],
          eval_metric='rmse',
          early_stopping_rounds=100,
          verbose=100)
rmspe(y_val, model.predict(x_val))

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.000809018
Did not meet early stopping. Best iteration is:
[20]	valid_0's rmse: 0.000739581


0.28815539507257315